In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd

from data_loader import DataLoader

In [7]:
# Usage
data_paths = ['../Data/Nantou/cn01.csv', 
              '../Data/Nantou/cn02.csv', 
              '../Data/Nantou/cn04.csv']
data_loader = DataLoader(data_paths)
data_loader.preprocess_data()

In [8]:
# Define Neural Network Model
class RegressionModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(RegressionModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [12]:
# Model Initialization
print(f"X_train shape: {data_loader.X_train.shape}")
print(f"y_train_totalmove shape: {data_loader.y_train_totalmove.shape}")


X_train shape: (459172, 10)
y_train_totalmove shape: (459172,)


In [15]:
# Model Initialization
input_dim = data_loader.X_train.shape[1]
output_dim = 1
model = RegressionModel(input_dim, output_dim)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 50
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch_X, batch_y in data_loader.train_loader:
        optimizer.zero_grad()
        predictions = model(batch_X)
        loss = criterion(predictions, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss / len(data_loader.train_loader):.4f}")

Epoch 1/50, Loss: 1440502.6948
Epoch 2/50, Loss: 1107936.1987
Epoch 3/50, Loss: 937860.6928
Epoch 4/50, Loss: 946040.2806
Epoch 5/50, Loss: 932133.7432
Epoch 6/50, Loss: 930686.7298
Epoch 7/50, Loss: 929727.4286
Epoch 8/50, Loss: 928680.2108
Epoch 9/50, Loss: 928105.3096
Epoch 10/50, Loss: 927726.0521
Epoch 11/50, Loss: 927379.5906
Epoch 12/50, Loss: 927248.0115
Epoch 13/50, Loss: 927153.0836
Epoch 14/50, Loss: 927134.4965
Epoch 15/50, Loss: 927127.2642
Epoch 16/50, Loss: 927124.0908
Epoch 17/50, Loss: 927123.6158
Epoch 18/50, Loss: 927123.3914
Epoch 19/50, Loss: 927123.1004
Epoch 20/50, Loss: 927122.5404
Epoch 21/50, Loss: 927122.9684
Epoch 22/50, Loss: 927122.9113
Epoch 23/50, Loss: 927122.7951
Epoch 24/50, Loss: 927122.3536
Epoch 25/50, Loss: 927122.6818
Epoch 26/50, Loss: 927122.4070
Epoch 27/50, Loss: 927122.3630
Epoch 28/50, Loss: 927122.5158
Epoch 29/50, Loss: 927122.1081
Epoch 30/50, Loss: 927122.3302
Epoch 31/50, Loss: 927122.1126
Epoch 32/50, Loss: 927122.0860
Epoch 33/50, Lo

In [ ]:
# Evaluation
model.eval()
with torch.no_grad():
    y_pred = model(data_loader.test_loader.X_test_tensor)
    mae = torch.mean(torch.abs(y_pred - data_loader.y_test_tensor), dim=0)
    rmse = torch.sqrt(torch.mean((y_pred - data_loader.y_test_tensor) ** 2, dim=0))

# Print results
results = pd.DataFrame({
    "Variable": ["TotalMove", "E", "N", "H"],
    "MAE": mae.numpy(),
    "RMSE": rmse.numpy()
})
print(results)


: 